# Plotly


Plotly has a Java sdk via JTableSaw project.

In [ ]:
//DEPS tech.tablesaw:tablesaw-jsplot:RELEASE

import tech.tablesaw.plotly.*;
import tech.tablesaw.plotly.api.*;
import tech.tablesaw.plotly.components.*;
import tech.tablesaw.plotly.traces.*;


In [ ]:
import org.dflib.jjava.jupyter.kernel.display.mime.MIMEType;

kernel().getRenderer().createRegistration(tech.tablesaw.plotly.components.Figure.class)
		.preferring(MIMEType.TEXT_HTML).register((figure, ctx) -> {
			ctx.renderIfRequested(MIMEType.TEXT_HTML, () -> {
				String id = UUID.randomUUID().toString().replace("-", "");
				boolean plotImage = System.getenv().containsKey("D2L_PLOT_IMAGE");

				figure.asJavascript(id);
				Map<String, Object> context = figure.getContext();
				StringBuilder html = new StringBuilder();
				html.append("<div id=\"").append(id).append("\"></div>\n");
				html.append("<script src=\"https://cdn.plot.ly/plotly-3.1.0.min.js\"></script>\n");
				html.append("<script>\n");
				if (!plotImage) {
					html.append("var ").append(context.get("targetName")).append(" = document.getElementById('")
							.append(id).append("');\n");
				} else {
					html.append("var ").append(context.get("targetName"))
							.append(" = document.createElement('div');\n");
					html.append("var ").append(context.get("targetName") + "_img")
							.append(" = document.getElementById('").append(id).append("_img');\n");
					String plotFunction = (String) context.get("plotFunction");
					String imgExt = ".then(function(gd){ Plotly.toImage(gd).then(function(url){ "
							+ context.get("targetName") + "_img.src = url; }); });";
					context.put("plotFunction", plotFunction.substring(0, plotFunction.length() - 1) + imgExt);
				}
				html.append(context.get("figure")).append('\n'); // your JS that defines data/layout/config
				html.append(context.get("plotFunction")).append('\n'); // should call Plotly.newPlot(...)
				html.append("</script>\n");

				return html.toString();
			});
		});

In [ ]:
import tech.tablesaw.api.*; // EDIT this import was missing in the original post
import tech.tablesaw.plotly.api.*;
import tech.tablesaw.plotly.components.*;

String[] animals = {"bear", "cat", "giraffe"};
double[] cuteness = {90.1, 84.3, 99.7};

Table cuteAnimals = Table.create("Cute Animals")
    .addColumns(
        StringColumn.create("Animal types", animals),
        DoubleColumn.create("rating", cuteness)
    );

Figure figure = AreaPlot.create("Cute animals", cuteAnimals, "Animal types", "rating");

display(figure);